# Cellular Automaton

Create an animated cellular automaton  

### You may need to add the path to ffmpeg to create the animation
Replace INSERT PATH HERE with your path, and uncomment the line

In [ ]:
import numpy as np
from numpy.random import seed
from numpy.random import rand
import matplotlib.pyplot as plt
# plt.rcParams['animation.ffmpeg_path'] = r'INSERT PATH HERE'
import matplotlib.cm as cm
from matplotlib import animation,rc
from IPython.display import HTML

### Universe Rules
Choose universe variables and settings.  
The default birth and death rules are for Conway's Game of Life

In [ ]:
select_variables = input("Would you like to choose universe variables? If No, default values will be used (Yes/No): ")

if select_variables == "Yes":
    universe_height = int(input("Enter universe height: "))
    universe_width = int(input("Enter universe width: "))
    overpop = int(input("Enter overpopulation limit (0-8):  "))
    underpop = int(input("Enter underpopulation limit (0-8): "))
    birth = int(input("Enter birth limit (0-8): "))
    afterglow = input("Enable afterglow effect? (Yes/No): ")
    wraparound  = input("Enable wraparound? (Yes/No): ")
    
else:
    universe_height = 30
    universe_width = 30
    overpop = 3
    underpop = 2
    birth = 3
    afterglow = "Yes"
    wraparound = "No"

### Patterns

Three standard patterns that can be added to the universe - the beacon oscillates in place, the glider travels diagonally down and right and the lightweight spaceship (LWSS) travels horizontally right.

More patterns can be added here (the array must not be jagged), and if added to pattern_list they can then be added to future universes with no further changes to the code

In [ ]:
beacon = [[1, 1, 0, 0],
          [1, 1, 0, 0],
          [0, 0, 1, 1],
          [0, 0, 1, 1]]

glider = [[0, 1, 0],
          [0, 0, 1],
          [1, 1, 1]]

LWSS = [[0, 0, 1, 0, 0, 0],
        [1, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1],
        [1, 0, 0, 0, 0, 1],
        [0, 1, 1, 1, 1, 1]]

pattern_list = {"Beacon": beacon, "Glider": glider, 
                "Lightweight Spaceship": LWSS}

### Universe Functions
Functions used to create and alter the universe

In [ ]:
def countNeighbours(x, y):
    num_neighbours = 0
    for i in (-1,0,1):
        for j in (-1,0,1):
            
            if (wraparound == "Yes"):
                if ((i != 0 or j != 0) and universe[(x+i)%universe_height, (y+j)%universe_width] == 1):
                    num_neighbours += 1
            else:

                if ((0 <= x+i < universe_height) and (0 <= y+j < universe_width)):
                    if ((i != 0 or j != 0) and (universe[x+i, y+j]) == 1):
                        num_neighbours += 1            

    return num_neighbours


def iterateUniverse():
    
    new_universe = np.zeros((universe_height, universe_width))                         
    new_universe[0:universe_height, 0:universe_width] = universe
    
    for x in range(0, universe_height):
        for y in range (0, universe_width):
            neighbours = countNeighbours(x, y)
            
            if afterglow == "Yes":
                if ((new_universe[x, y] == 1) and (neighbours < underpop or neighbours > overpop)):
                    new_universe[x, y] = 0.5
                
                if ((new_universe[x, y] < 1) and (neighbours is birth)):
                    new_universe[x, y] = 1
                elif (0 < new_universe[x, y] < 1):
                    new_universe[x, y] -= 0.02
            
            else:
                if ((new_universe[x, y] == 1) and (neighbours < underpop or neighbours > overpop)):
                    new_universe[x, y] = 0
                
                if ((new_universe[x, y] == 0) and (neighbours is birth)):
                    new_universe[x, y] = 1                
            
                
    
    return new_universe

    
def randomUniverse(universe_density):
    
    for x in range(universe_height):
        for y in range(universe_width):
            if np.random.random() < universe_density:
                universe[x][y] = 1
                
                
def insertPattern(number_of_patterns):
    
    for n in range(number_of_patterns):
        print("Which pattern would you like to add? Choose one of:", *pattern_list, sep = "\n")
        pattern = input()
        pattern_x = int(input("Enter x coordinate for the pattern: "))
        pattern_y = int(input("Enter y coordinate for the pattern: "))
        
        pattern_length = len(pattern_list[pattern])
        pattern_width = len(pattern_list[pattern][0])
        universe[pattern_x:(pattern_x + pattern_length), pattern_y:(pattern_y + pattern_width)] = pattern_list[pattern]
        


### Animation Functions
Functions needed for FuncAnimations

In [ ]:
def init():
    return (im,)

def animate(i):
    if i is 0:
        return [im]
    else:
        universe[0:universe_height, 0:universe_width] = iterateUniverse()
        im.set_array(universe)
        return [im]

### Create the Universe
Run to create the universe with two options:  
Randomly set cells to alive with a chosen probability  
or  
Enter created patterns into an empty universe

In [ ]:
universe = np.zeros((universe_height, universe_width))

universe_type = input("Random universe or empty universe with patterns? (Random/Empty): ")

if universe_type == "Empty":
    insertPattern(int(input("How many patterns would you like to add?")))

else:
    randomUniverse(float(input("Enter universe density (0-1): ")))

### Figure Creation
Create the base figure for your animation

In [ ]:
print("Initial conditions of the universe: ")
fig = plt.figure()
fig = plt.figure(figsize = (10,10))
plt.axis("off")
im = plt.imshow(universe, cmap = 'Greys')
plt.show()

### Animation
Run to create the animation.  

In [ ]:
anim = animation.FuncAnimation(fig, func=animate, init_func=init, frames=200, interval = 100, blit=True)
HTML(anim.to_html5_video())